**LLM기반 정보 추출**     
LLM을 활용하여 뉴스 기사 중에서 `중요한 기사만 골라내고`    
몽고db 컬렉션 `yna+preprocessed_v2`에 있는 문서에서 아래의 정보를 추출하는것이 목표:
| 항목             | 설명                                 |
| -------------- | ---------------------------------- |
| `기사 날짜`        | pubDate (기존 필드)                    |
| `기사 카테고리`      | 기존 category 필드                     |
| `사건 발생일자`      | 본문 내 상대 표현을 변환                     |
| `사건 장소`        | ex. "평양", "청진항" 등                  |
| `핵심 인물`        | 김정은, 라브로프 등                        |
| `기사 요약`        | 요약된 2\~3줄                          |
| `기사 URL`       | 기존 url 필드                          |
| `핵심 키워드`       | "ICBM", "군사 협력", "라브로프", "무기 이전" 등 |
| ✅ **중요 기사 여부** | 중요하면 추출, 아니면 제외                    |


In [ ]:
# DB에서 데이터 불러오기
from pymongo import MongoClient, errors
import datetime

# --- MongoDB 연결 설정 ---
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "polaris"
SOURCE_COLLECTION_NAME = "yna_preprocessed_v3" 

# --- 날짜 범위 설정 ---
# 시작 날짜: 년 월 일 시 분 초 
start_date = datetime.datetime(2024, 5, 1, 0, 0, 0)
# 종료 날짜: 년 월 일 시 분 초 
end_date = datetime.datetime(2024, 5, 2, 23, 59, 59)

raw_mongo_docs = []
client = None
try:
    client = MongoClient(MONGO_URI)
    db = client[DB_NAME]
    source_collection = db[SOURCE_COLLECTION_NAME]

    print(f"MongoDB에 연결되었습니다. '{SOURCE_COLLECTION_NAME}' 컬렉션에서 특정 날짜 범위의 문서를 로드합니다...")

    # MongoDB 쿼리: pubDate 필드가 start_date 이상이고 end_date 이하인 문서
    date_range_query = {
        "pubDate": {
            "$gte": start_date,
            "$lte": end_date
        }
    }

    # pubDate를 기준으로 오름차순 정렬하여 문서 조회
    cursor = source_collection.find(date_range_query).sort('pubDate', 1) 
    
    for doc in cursor:
        raw_mongo_docs.append(doc)
    
    print(f"총 {len(raw_mongo_docs)}개의 문서를 MongoDB에서 로드했습니다.")
    print(f"날짜 범위: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")

except errors.ConnectionFailure as e:
    print(f"MongoDB 연결 오류: {e}")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
finally:
    if client:
        client.close()
        print("MongoDB 연결을 닫았습니다.")


In [ ]:
# 라마 인덱스 문서 변환하기
from llama_index.core import Document
import json

all_llama_docs = []
print("로드된 문서를 LlamaIndex Document로 변환 중...")
    
for doc in raw_mongo_docs:
    content_text = doc.get("content", "")
    if not isinstance(content_text, str):
        content_text = str(content_text) if content_text is not None else ""

    title_text = doc.get("title", "")
    if not isinstance(title_text, str):
        title_text = str(title_text) if title_text is not None else ""

    llama_doc = Document(
        text=content_text, 
        metadata={
            "title": title_text, 
            "pubDate": str(doc.get("pubDate", "")), 
            "url": doc.get("url", ""),
            "category": doc.get("category")
        }
    )
    all_llama_docs.append(llama_doc)
    
print(f"총 {len(all_llama_docs)}개의 문서를 LlamaIndex Document로 변환했습니다.")

In [ ]:
# # LlamaIndex Node로 청킹하기

# from llama_index.core.node_parser import SentenceSplitter

# if 'all_llama_docs' not in locals() or not all_llama_docs:
#     print("오류: 'all_llama_docs'가 비어 있거나 생성되지 않았습니다.")
# else:
#     print("LlamaIndex Document들을 Node(청크)로 분할 중...")
#     # LLM 모델 없이 텍스트 기반 청킹만 수행
#     node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20) # 청크 사이즈 조절 가능
#     all_nodes = node_parser.get_nodes_from_documents(all_llama_docs)
    
#     print(f"총 {len(all_nodes)}개의 노드(청크)를 생성했습니다.")


In [ ]:
# 생성된 노드의 첫 텍스트 미리보기
    # if all_nodes:
    #    print(f"첫 번째 Node 내용 미리보기: {all_nodes[0].text[:100]}...")

In [ ]:
# Node들을 JSON 파일로 저장하기

import json

if 'all_llama_docs' not in locals() or not all_llama_docs:
    print("오류: 'all_llama_docs'가 비어 있거나 생성되지 않았습니다.")
else:
    print("생성된 노드(청크)들을 직렬화하여 파일로 저장 중...")

    # 각 노드를 딕셔너리로 변환하여 JSON으로 저장
    serializable_nodes = [node.to_dict() for node in all_llama_docs]
    
    output_filename = "prepared_nodes_for_llm_vm.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(serializable_nodes, f, ensure_ascii=False, indent=2)

    print(f"변환된 노드 데이터가 '{output_filename}' 파일로 성공적으로 저장되었습니다.")

In [ ]:
# # Node들을 JSON 파일로 저장하기(청킹 할 떄)

import json
# LlamaIndex 0.10.x 이상에서 Node 객체의 to_dict()를 위해 필요할 수 있습니다.
# from llama_index.core.schema import Node 

# LlamaIndex Node 객체의 to_dict() 결과에서 불필요한 필드를 정리하는 함수
def clean_llama_node_dict(data: dict) -> dict:
    cleaned_data = data.copy() # 원본 딕셔너리를 직접 수정하지 않기 위해 복사

    # 1. 최상위 레벨의 불필요한 필드 제거
    # 'embedding' 필드는 나중에 생성되므로 저장 시에는 제거
    if 'embedding' in cleaned_data and cleaned_data['embedding'] is None:
        del cleaned_data['embedding']
    
    # 텍스트 노드에서는 이미지, 오디오, 비디오 리소스가 없음
    if 'image_resource' in cleaned_data and cleaned_data['image_resource'] is None:
        del cleaned_data['image_resource']
    if 'audio_resource' in cleaned_data and cleaned_data['audio_resource'] is None:
        del cleaned_data['audio_resource']
    if 'video_resource' in cleaned_data and cleaned_data['video_resource'] is None:
        del cleaned_data['video_resource']

    # 2. 'text_resource' 내부의 불필요한 필드 제거
    if 'text_resource' in cleaned_data and isinstance(cleaned_data['text_resource'], dict):
        tr = cleaned_data['text_resource']
        if 'embeddings' in tr and tr['embeddings'] is None:
            del tr['embeddings']
        if 'path' in tr and tr['path'] is None:
            del tr['path']
        # text_resource.url은 원본 metadata.url과 다를 수 있으므로, null이면 제거
        if 'url' in tr and tr['url'] is None: 
            del tr['url']
        if 'mimetype' in tr and tr['mimetype'] is None:
            del tr['mimetype']
    return cleaned_data

if 'all_nodes' not in locals() or not all_nodes:
    print("오류: 'all_nodes'가 비어 있거나 생성되지 않았습니다. 이전 구문을 먼저 실행해주세요.")
else:
    print("생성된 노드(청크)들을 직렬화하여 파일로 저장 중...")

    serializable_nodes = []
    for node in all_nodes:
        node_dict = node.to_dict() # Node 객체를 딕셔너리로 변환
        cleaned_node_dict = clean_llama_node_dict(node_dict) # 불필요한 필드 제거
        serializable_nodes.append(cleaned_node_dict)
    
    output_filename = "prepared_nodes_for_llm_vm_6_30.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(serializable_nodes, f, ensure_ascii=False, indent=2)

    print(f"변환된 노드 데이터가 '{output_filename}' 파일로 성공적으로 저장되었습니다.")